In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from multiprocessing import Pool
from pathlib import Path
from subprocess import Popen, PIPE
import time

import copy
import itertools
import time
from pathlib import Path
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, models, transforms

from multiprocessing import cpu_count
from multiprocessing.dummy import Pool
import numpy as np
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer
import tqdm
from PIL import Image

import matplotlib as mpl
mpl_params = {
    'figure.figsize': (10, 5),
    'figure.dpi': 300,
}
from matplotlib import pyplot as plt
mpl.rcParams.update(mpl_params)

import seaborn as sns
sns.set()

In [2]:
test = Path('../input/human-protein-atlas-image-classification/test/')
train = Path('../input/human-protein-atlas-image-classification/train/')
hpa = Path('../input/HPAv18/')

test_df = pd.read_csv('../input/human-protein-atlas-image-classification/sample_submission.csv')
train_df = pd.read_csv('../input/human-protein-atlas-image-classification/train.csv')
hpa_df = pd.read_csv('../HPAv18RBGY_wodpl.csv')

In [3]:
train_df.iloc[0]

Id        00070df0-bbc3-11e8-b2bc-ac1f6b6435d0
Target                                    16 0
Name: 0, dtype: object

In [4]:
hpa_df.iloc[0]

Id        10580_1610_C1_1
Target       13 25 0 2 21
Name: 0, dtype: object

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

In [6]:
def load_hpa(id_):
    dir_ = hpa
    rgb = np.array(Image.open(dir_ / f'{id_}.png'), np.uint8)
    return {id_: transform(rgb)}

def load_train(id_):
    dir_ = train
    colors = ['red', 'green', 'blue']
    image_paths = [dir_ / f'{id_}_{c}.png' for c in colors]
    r, g, b = map(Image.open, image_paths)
    rgb = np.stack([r, g, b], axis=2)
    return {id_: transform(rgb)}

def load_test(id_):
    dir_ = test
    colors = ['red', 'green', 'blue']
    image_paths = [dir_ / f'{id_}_{c}.png' for c in colors]
    r, g, b = map(Image.open, image_paths)
    rgb = np.stack([r, g, b], axis=2)
    return {id_: transform(rgb)}

In [7]:
p = Pool()

In [8]:
%%time
test_stack = p.map(load_test, test_df.Id)
test_stack = {k: v for d in test_stack for k, v in d.items()}

CPU times: user 2min 54s, sys: 58.1 s, total: 3min 52s
Wall time: 46 s


In [ ]:
%%time
np.savez_compressed('test.npz', **test_stack)